In [75]:
# !pip install torch==1.7.1
# !pip install transformers==4.11.3
# !pip install huggingface-hub==0.0.19
# !pip install datasets==1.5.0

In [1]:
import json
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from tqdm import trange

from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    BertModel, RobertaModel,
    BertPreTrainedModel,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments,
)
from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from typing import List
from torch.utils.data import Sampler

In [2]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    random.seed(random_seed)
    np.random.seed(random_seed)
    
set_seed(42) # magic number :)

In [3]:
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.7.1].
device:[cuda:0].


## Training

In [4]:
class BertEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
        classifier_dropout=(
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = torch.nn.Dropout(classifier_dropout)
        self.linear = torch.nn.Linear(config.hidden_size, 1)
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        output = self.linear(pooled_output)
        return output

In [5]:
dataset = load_from_disk('/opt/ml/data/train_dataset')
train_dataset = dataset['train']

In [6]:
class CustomSampler(Sampler) :
    def __init__(self, data_source, batch_size) :
        self.data_source = data_source
        self.batch_size = batch_size

    def __iter__(self) :
        n = len(self.data_source)
        index_list = []
        while True :
            out = True
            for i in range(self.batch_size) :
                tmp_data = random.randint(0, n-1)
                index_list.append(tmp_data)
            for f, s in zip(index_list, index_list[1:]) :
                if abs(s-f) <= 2 :
                    out = False
            if out == True :
                break

        while True : # 추가 삽입
            tmp_data = random.randint(0, n-1)
            if (tmp_data not in index_list) and \
                (abs(tmp_data-index_list[-i]) > 2 for i in range(1,self.batch_size+1)) \
            : 
                index_list.append(tmp_data)
            if len(index_list) == n :
                break
        return iter(index_list)

    def __len__(self) :
        return len(self.data_source)

In [7]:
# Anwer
class DenseRetrieval:
    def __init__(self,
        args,
        dataset,
        tokenizer,
        cross_encoder,
        sampler
    ):
        """
        학습과 추론에 사용될 여러 셋업을 마쳐봅시다.
        """

        self.args = args
        self.dataset = dataset

        self.tokenizer = tokenizer
        self.cross_encoder = cross_encoder
        self.sampler = sampler

    def train(self, args=None, tokenizer = None):
        if args is None:
            args = self.args
        if tokenizer is None :
            tokenizer = self.tokenizer
        
        tokenized_examples = tokenizer(
            self.dataset['question'],
            self.dataset['context'],
            truncation="only_second",
            max_length=512,
            stride=128,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            # return_token_type_ids=False,  # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
            padding="max_length",
            return_tensors='pt'
        )

        train_dataset = TensorDataset(
            tokenized_examples['input_ids'],
            tokenized_examples['attention_mask'],
            tokenized_examples['token_type_ids']
        )

        sampler = self.sampler(train_dataset, args.per_device_train_batch_size)
        train_dataloader = DataLoader(train_dataset,
                                      batch_size=args.per_device_train_batch_size,
                                      sampler = sampler,
                                      drop_last = True)
                                      
        no_decay = ["bias" ,"LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {"params": [p for n, p in self.cross_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.cross_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
        ]
        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=args.learning_rate,
            # eps=args.adam_epsilon
        )

        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)
        
        self.cross_encoder.zero_grad()
        
        train_iterator = trange(int(args.num_train_epochs), desc="Epoch")
        self.cross_encoder.train()
        for epoch, _ in enumerate(train_iterator) :
            epoch_iterator = tqdm(train_dataloader, desc = 'Iteration')
            losses = 0
            for step, batch in enumerate(epoch_iterator) :
                # if torch.cuda.is_available() :
                #     batch = tuple(t.cuda() for t in batch)
                
                cross_inputs = {
                    'input_ids': batch[0],
                    'attention_mask' : batch[1],
                    'token_type_ids' : batch[2]
                }
                for k in cross_inputs.keys() :
                    cross_inputs[k] = cross_inputs[k].tolist()

                new_input_ids = []
                new_attention_mask = []
                new_token_type_ids = []
                for i in range(len(cross_inputs['input_ids'])) :
                    sep_index = cross_inputs['input_ids'][i].index(3) # [SEP] token의 index

                    for j in range(len(cross_inputs['input_ids'])) :
                        query_id = cross_inputs['input_ids'][i][:sep_index]
                        query_att = cross_inputs['attention_mask'][i][:sep_index]
                        query_tok = cross_inputs['token_type_ids'][i][:sep_index]
        
                        context_id = cross_inputs['input_ids'][j][sep_index:]
                        context_att = cross_inputs['attention_mask'][j][sep_index:]
                        context_tok = cross_inputs['token_type_ids'][j][sep_index:]
                        query_id.extend(context_id)
                        query_att.extend(context_att)
                        query_tok.extend(context_tok)
                        new_input_ids.append(query_id)
                        new_attention_mask.append(query_att)
                        new_token_type_ids.append(query_tok)

                change_cross_inputs = {
                    'input_ids' : torch.tensor(new_input_ids).to('cuda'),
                    'attention_mask' : torch.tensor(new_attention_mask).to('cuda'),
                    'token_type_ids' : torch.tensor(new_token_type_ids).to('cuda')
                }

                cross_output = self.cross_encoder(**change_cross_inputs)
                cross_output = cross_output.view(-1, args.per_device_train_batch_size)
                targets = torch.arange(0, args.per_device_train_batch_size).long()
                                
                if torch.cuda.is_available():
                    targets = targets.to('cuda')

                score = F.log_softmax(cross_output, dim = 1)
                loss = F.nll_loss(score, targets)
                #########################No ACCUMULATION#########################
                # losses += loss.item()
                # if step % 100 == 0 :
                #     print(f'{epoch}epoch loss: {losses/(step+1)}') # Accumulation할 경우 주석처리
                
                # self.cross_encoder.zero_grad()
                # loss.backward()
                # optimizer.step()
                # scheduler.step()
                #################################################################

                #############################ACCUMULATION#########################
                loss.backward()
                if (step+1) % args.gradient_accumulation_steps == 0 :
                    optimizer.step()
                    scheduler.step()
                    self.cross_encoder.zero_grad()

                losses += loss.item()
                if (step+1) % 100 == 0 :
                    train_loss = losses / 100
                    print(f'training loss: {train_loss:4.4}')
                    losses = 0
                ##################################################################
        
        return self.cross_encoder

In [8]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=40,
    weight_decay=0.01
)
model_checkpoint = "klue/bert-base"

# 혹시 위에서 사용한 encoder가 있다면 주석처리 후 진행해주세요 (CUDA ...)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
cross_encoder = BertEncoder.from_pretrained(model_checkpoint).to('cuda')

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertEncoder were not initialized from the model checkpoint at klue/bert-base and are newly initialized: 

In [9]:
# Retriever는 아래와 같이 사용할 수 있도록 코드를 짜봅시다.
retriever = DenseRetrieval(
    args=args,
    dataset=train_dataset,
    tokenizer=tokenizer,
    cross_encoder=cross_encoder,
    sampler = CustomSampler
)
c_encoder = retriever.train()

Epoch:   0%|          | 0/40 [00:00<?, ?it/s]

training loss: 0.3505
training loss: 0.0798
training loss: 0.0634
training loss: 0.02339
training loss: 0.01976
training loss: 0.01066
training loss: 0.01028
training loss: 0.01023
training loss: 0.01002
training loss: 0.007944
training loss: 0.009278
training loss: 0.008298
training loss: 0.005324
training loss: 0.007314


Epoch:   2%|▎         | 1/40 [12:06<7:51:55, 726.04s/it]

training loss: 0.007787
training loss: 0.002427
training loss: 0.00171
training loss: 0.0009209
training loss: 0.007005
training loss: 0.003466
training loss: 0.001762
training loss: 0.001404
training loss: 0.001284
training loss: 0.0008469
training loss: 0.0007349
training loss: 0.008568
training loss: 0.002439
training loss: 0.004033


Epoch:   5%|▌         | 2/40 [24:11<7:39:42, 725.87s/it]

...

Epoch:  95%|█████████▌| 38/40 [7:43:03<26:49, 804.54s/it]

training loss: 0.002083
training loss: 0.003962
training loss: 0.0004586
training loss: 2.468e-05
training loss: 0.001955
training loss: 0.003146
training loss: 5.406e-05
training loss: 4.448e-06
training loss: 4.7e-06
training loss: 0.005023
training loss: 5.499e-06
training loss: 9.262e-06
training loss: 0.0002058
training loss: 1.164e-05


Epoch:  98%|█████████▊| 39/40 [7:55:30<13:07, 787.15s/it]

training loss: 4.615e-05
training loss: 2.516e-05
training loss: 5.389e-06
training loss: 2.925e-06
training loss: 1.48e-05
training loss: 9.347e-06
training loss: 0.0001307
training loss: 9.974e-06
training loss: 0.001009
training loss: 2.184e-05
training loss: 5.502e-05
training loss: 1.904e-05
training loss: 3.772e-05
training loss: 3.033e-05


Epoch: 100%|██████████| 40/40 [8:07:35<00:00, 731.38s/it]

In [10]:
torch.save(c_encoder, '/opt/ml/custom/c_encoder_e40_b16.pt')

## 실험

In [11]:
valid_corpus = list(set([example['context'] for example in dataset['validation']]))[:10]
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

print(query)
print(ground_truth)

볼드윈이 "당신들은 나를 야유합니까?"라는 말을 한 연도는?
퇴임에서 볼드윈의 세월은 조용하였다. 네빌 체임벌린이 사망하면서 전쟁 이전의 유화 정책에서 볼드윈의 지각된 부분은 제2차 세계 대전이 일어난 동안과 그 후에 그를 인기없는 인물로 만들었다. 신문의 캠페인은 그를 전쟁 생산에 자신의 시골 저택의 철문을 기부하지 않은 것으로 사냥하였다. 전쟁이 일어난 동안 윈스턴 처칠은 에이먼 데 벌레라의 아일랜드의 지속적인 중립을 향한 더욱 힘든 경향을 취하는 영국의 조언에 그를 단 한번 상담하였다.\n\n1945년 6월 부인 루시 여사가 사망하였다. 이제 볼드윈 자신은 관절염을 겪어 걸어다는 데 지팡이가 필요하였다. 조지 5세의 동상의 공개식에 1947년 런던에서 자신의 최종 공개적인 출연을 이루었다. 관중들은 전직 총리를 알아주어 그를 응원하였으나 이 당시 볼드윈은 귀머거리였고, 그들에게 "당신들은 나를 야유합니까?"라고 의문하였다. 1930년 케임브리지 대학교의 총장으로 만들어진 그는 1947년 12월 14일 80세의 나이에 우스터셔주 스투어포트온세번 근처 애슬리홀에서 수면 중 자신의 사망까지 이 수용력에 지속하였다. 그는 화장되었고, 그의 재는 우스터 대성당에 안치되었다.


In [12]:
with torch.no_grad() :
    c_encoder.eval()
    
    score_list = []
    for i in range(len(valid_corpus)) :
        passage = valid_corpus[i]
        tokenized_examples = tokenizer(
            query,
            passage,
            truncation="only_second",
            max_length=512,
            stride=128,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            #return_token_type_ids=False,  # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
            padding="max_length",
            return_tensors='pt'
        )

        score = 0
        for i in range(len(tokenized_examples['input_ids'])) :
            c_input = {
                'input_ids' : torch.tensor(tokenized_examples['input_ids'][i].unsqueeze(dim=0)).to('cuda'),
                'attention_mask' : torch.tensor(tokenized_examples['attention_mask'][i].unsqueeze(dim=0)).to('cuda'),
                'token_type_ids' : torch.tensor(tokenized_examples['token_type_ids'][i].unsqueeze(dim=0)).to('cuda')
            }
            tmp_score = c_encoder(**c_input).to('cpu')
            score += tmp_score
        score = score / len(tokenized_examples['input_ids'])
        score_list.append(score)
    sort_result = torch.sort(torch.tensor(score_list), descending=True)

    scores, index_list = sort_result[0], sort_result[1]

<ipython-input-12-411eda001f16>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids' : torch.tensor(tokenized_examples['input_ids'][i].unsqueeze(dim=0)).to('cuda'),
<ipython-input-12-411eda001f16>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask' : torch.tensor(tokenized_examples['attention_mask'][i].unsqueeze(dim=0)).to('cuda'),
<ipython-input-12-411eda001f16>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids' : torch.tensor(tokenized_examples['token_type_ids'][i].unsqueeze(dim=0)).to('cuda')


In [13]:
k = 5
print("[Search query]\n", query, "\n")
print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, scores[i]))
  print(valid_corpus[index_list[i]])

[Search query]
 볼드윈이 "당신들은 나를 야유합니까?"라는 말을 한 연도는? 

[Ground truth passage]
퇴임에서 볼드윈의 세월은 조용하였다. 네빌 체임벌린이 사망하면서 전쟁 이전의 유화 정책에서 볼드윈의 지각된 부분은 제2차 세계 대전이 일어난 동안과 그 후에 그를 인기없는 인물로 만들었다. 신문의 캠페인은 그를 전쟁 생산에 자신의 시골 저택의 철문을 기부하지 않은 것으로 사냥하였다. 전쟁이 일어난 동안 윈스턴 처칠은 에이먼 데 벌레라의 아일랜드의 지속적인 중립을 향한 더욱 힘든 경향을 취하는 영국의 조언에 그를 단 한번 상담하였다.\n\n1945년 6월 부인 루시 여사가 사망하였다. 이제 볼드윈 자신은 관절염을 겪어 걸어다는 데 지팡이가 필요하였다. 조지 5세의 동상의 공개식에 1947년 런던에서 자신의 최종 공개적인 출연을 이루었다. 관중들은 전직 총리를 알아주어 그를 응원하였으나 이 당시 볼드윈은 귀머거리였고, 그들에게 "당신들은 나를 야유합니까?"라고 의문하였다. 1930년 케임브리지 대학교의 총장으로 만들어진 그는 1947년 12월 14일 80세의 나이에 우스터셔주 스투어포트온세번 근처 애슬리홀에서 수면 중 자신의 사망까지 이 수용력에 지속하였다. 그는 화장되었고, 그의 재는 우스터 대성당에 안치되었다. 

Top-1 passage with score 9.6913
퇴임에서 볼드윈의 세월은 조용하였다. 네빌 체임벌린이 사망하면서 전쟁 이전의 유화 정책에서 볼드윈의 지각된 부분은 제2차 세계 대전이 일어난 동안과 그 후에 그를 인기없는 인물로 만들었다. 신문의 캠페인은 그를 전쟁 생산에 자신의 시골 저택의 철문을 기부하지 않은 것으로 사냥하였다. 전쟁이 일어난 동안 윈스턴 처칠은 에이먼 데 벌레라의 아일랜드의 지속적인 중립을 향한 더욱 힘든 경향을 취하는 영국의 조언에 그를 단 한번 상담하였다.\n\n1945년 6월 부인 루시 여사가 사망하였다. 이제 볼드윈 자신은 관절염을 겪어 걸어다는 데 지팡이가 필요하였다. 조지 5세의

In [28]:
index_list

tensor([10,  0,  4,  1,  5,  9,  6,  3,  7,  2,  8])

In [27]:
tokenized_examples['input_ids'][0].unsqueeze(dim=0).shape

torch.Size([1, 512])

In [ ]:
top_k_index_list = []
for i in range(len(index_list)) :
    temp = index_list[i][:k]
    top_k_index_list.appedn(temp)

## 실제

In [30]:
with open('/opt/ml/data/wikipedia_documents.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)

corpus = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)  # set 은 매번 순서가 바뀌므로

In [32]:
question_data = dataset['validation']['question']
with torch.no_grad() :
    c_encoder.eval()

    result_scores = []
    result_indices = []
    for i in tqdm(range(len(question_data))) :
        question = question_data[i]

        question_score = []
        for i in tqdm(range(len(corpus))) :
            passage = corpus[i]
            tokenized_examples = tokenizer(
                question,
                passage,
                truncation="only_second",
                max_length=512,
                stride=128,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                #return_token_type_ids=False,  # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
                padding="max_length",
                return_tensors='pt'
            )

            score = 0
            for i in range(len(tokenized_examples['input_ids'])) :
                c_input = {
                    'input_ids' : torch.tensor(tokenized_examples['input_ids'][i].unsqueeze(dim=0)).to('cuda'),
                    'attention_mask' : torch.tensor(tokenized_examples['attention_mask'][i].unsqueeze(dim=0)).to('cuda'),
                    'token_type_ids' : torch.tensor(tokenized_examples['token_type_ids'][i].unsqueeze(dim=0)).to('cuda')
                }
                tmp_score = c_encoder(**c_input).to('cpu')
                score += tmp_score
            score = score / len(tokenized_examples['input_ids'])
            question_score.append(score)

        sort_result = torch.sort(torch.tensor(question_score), descending=True)
        scores, index_list = sort_result[0], sort_result[1]

        result_scores.append(scores)
        result_indices.append(index_list)

<ipython-input-32-a97d169c2490>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids' : torch.tensor(tokenized_examples['input_ids'][i].unsqueeze(dim=0)).to('cuda'),
<ipython-input-32-a97d169c2490>:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask' : torch.tensor(tokenized_examples['attention_mask'][i].unsqueeze(dim=0)).to('cuda'),
<ipython-input-32-a97d169c2490>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids' : torch.tensor(tokenized_examples['token_type_ids'][i].unsqueeze(dim=0)).to('cuda')


KeyboardInterrupt: 

In [ ]:
top_k_index_list = []
for i in range(len(index_list)) :
    temp = index_list[i][:k]
    top_k_index_list.appedn(temp)

In [ ]:
total = []
for idx, example in enumerate(
        tqdm(dataset['validation'], desc="Dense retrieval: ")
    ):
        tmp = {
            # Query와 해당 id를 반환합니다.
            "question": example["question"],
            "id": example["id"],
            # Retrieve한 Passage의 id, context를 반환합니다.
            "context_id": top_k_index_list[idx],
            "context": " ".join(  # 기존에는 ' '.join()
                [corpus[pid] for pid in top_k_index_list[idx]]
            ),
        }
        if "context" in example.keys() and "answers" in example.keys():
            # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
            tmp["original_context"] = example["context"]
            tmp["answers"] = example["answers"]
        total.append(tmp)

cqas_100 = pd.DataFrame(total)

## Elastic

In [11]:
dataset = load_from_disk('/opt/ml/data/train_dataset')
train_dataset = dataset['train']

In [12]:
data = pd.read_csv('/opt/ml/custom/top100_wikipedia.csv')

In [13]:
doc_indices = []
for i in range(len(data)) :
    tmp = eval(data['document_id'][i])
    doc_indices.append(tmp)

In [14]:
with open('/opt/ml/data/wikipedia_documents.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)

corpus = []
for v in wiki.values() :
    corpus.append(v['text'])

In [16]:
class BertEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
        classifier_dropout=(
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = torch.nn.Dropout(classifier_dropout)
        self.linear = torch.nn.Linear(config.hidden_size, 1)
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        output = self.linear(pooled_output)
        return output

In [12]:
c_encoder = torch.load('/opt/ml/custom/c_encoder_e20.pt')

In [15]:
model_checkpoint = "klue/bert-base"
# 혹시 위에서 사용한 encoder가 있다면 주석처리 후 진행해주세요 (CUDA ...)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
question_data = dataset['validation']['question']
with torch.no_grad() : 
    c_encoder.eval()

    result_scores = []
    result_indices = []
    for i in tqdm(range(len(question_data))) :
        question = question_data[i]
        question_score = []
        for indice in tqdm(doc_indices[i]) :
            passage = corpus[indice]
            tokenized_examples = tokenizer(
                question,
                passage,
                truncation="only_second",
                max_length=512,
                stride=128,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                #return_token_type_ids=False,  # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
                padding="max_length",
                return_tensors='pt'
            )
            score = 0
            for i in range(len(tokenized_examples['input_ids'])) :
                c_input = {
                    'input_ids' : torch.tensor(tokenized_examples['input_ids'][i].unsqueeze(dim=0)).to('cuda'),
                    'attention_mask' : torch.tensor(tokenized_examples['attention_mask'][i].unsqueeze(dim=0)).to('cuda'),
                    'token_type_ids' : torch.tensor(tokenized_examples['token_type_ids'][i].unsqueeze(dim=0)).to('cuda')
                }
                tmp_score = c_encoder(**c_input).to('cpu')
                score += tmp_score
            score = score / len(tokenized_examples['input_ids'])
            question_score.append(score)
        sort_result = torch.sort(torch.tensor(question_score), descending=True)
        scores, index_list = sort_result[0], sort_result[1]

        result_scores.append(scores.tolist())
        result_indices.append(index_list.tolist())        

In [33]:
final_indices = []
for i in range(len(doc_indices)) :
    t_list = [doc_indices[i][result_indices[i][k]] for k in range(7)]
    final_indices.append(t_list)

In [34]:
total = []
for idx, example in enumerate(
        tqdm(dataset['validation'], desc="Dense retrieval: ")
    ):
        tmp = {
            # Query와 해당 id를 반환합니다.
            "question": example["question"],
            "id": example["id"],
            # Retrieve한 Passage의 id, context를 반환합니다.
            "context_id": final_indices[idx],
            "context": " ".join(  # 기존에는 ' '.join()
                [corpus[pid] for pid in final_indices[idx]]
            ),
        }
        if "context" in example.keys() and "answers" in example.keys():
            # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
            tmp["original_context"] = example["context"]
            tmp["answers"] = example["answers"]
        total.append(tmp)

cqas_50 = pd.DataFrame(total)

In [35]:
correct_length = []
for i in range(len(cqas_50)) :
    if cqas_50['original_context'][i] in cqas_50['context'][i] :
        correct_length.append(i)
print(len(correct_length) / len(dataset['validation']))

0.875


In [36]:
cqas_50.to_csv('b16_special_shuffle_elastic_ce40_t7.csv', index = False)

## Test

In [7]:
dataset = load_from_disk('/opt/ml/data/test_dataset')

In [8]:
data = pd.read_csv('/opt/ml/custom/test_elastic_top100.csv')

doc_indices = []
for i in range(len(data)) :
    tmp = eval(data['document_id'][i])
    doc_indices.append(tmp)

In [9]:
# test에 대해서만 실행
for i in tqdm(range(len(doc_indices))) :
    for j in range(len(doc_indices[i])) :
        doc_indices[i][j] = int(doc_indices[i][j])

In [10]:
with open('/opt/ml/data/wikipedia_documents.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)

corpus = []
for v in wiki.values() :
    corpus.append(v['text'])

In [11]:
class BertEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
        classifier_dropout=(
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = torch.nn.Dropout(classifier_dropout)
        self.linear = torch.nn.Linear(config.hidden_size, 1)
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        output = self.linear(pooled_output)
        return output

In [12]:
c_encoder = torch.load('/opt/ml/custom/c_encoder_e40_b16.pt')
model_checkpoint = "klue/bert-base"
# 혹시 위에서 사용한 encoder가 있다면 주석처리 후 진행해주세요 (CUDA ...)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
question_data = dataset['validation']['question']
with torch.no_grad() : 
    c_encoder.eval()

    result_scores = []
    result_indices = []
    for i in tqdm(range(len(question_data))) :
        question = question_data[i]
        question_score = []
        for indice in tqdm(doc_indices[i]) :
            passage = corpus[int(indice)]
            tokenized_examples = tokenizer(
                question,
                passage,
                truncation="only_second",
                max_length=512,
                stride=128,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                #return_token_type_ids=False,  # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
                padding="max_length",
                return_tensors='pt'
            )
            score = 0
            for i in range(len(tokenized_examples['input_ids'])) :
                c_input = {
                    'input_ids' : torch.tensor(tokenized_examples['input_ids'][i].unsqueeze(dim=0)).to('cuda'),
                    'attention_mask' : torch.tensor(tokenized_examples['attention_mask'][i].unsqueeze(dim=0)).to('cuda'),
                    'token_type_ids' : torch.tensor(tokenized_examples['token_type_ids'][i].unsqueeze(dim=0)).to('cuda')
                }
                tmp_score = c_encoder(**c_input).to('cpu')
                score += tmp_score
            score = score / len(tokenized_examples['input_ids'])
            question_score.append(score)
        sort_result = torch.sort(torch.tensor(question_score), descending=True)
        scores, index_list = sort_result[0], sort_result[1]

        result_scores.append(scores.tolist())
        result_indices.append(index_list.tolist())        

### result_indices 저장 및 불러오기

In [26]:
import csv
with open('listfile.csv', 'w', newline='') as f: 
    writer = csv.writer(f)
    writer.writerow(result_indices)

In [29]:
with open('listfile.csv', 'r', encoding='utf-8') as f:
    rdr = csv.reader(f)
    for i, line in enumerate(rdr) :
        if i == 0 :
            kk = line

### 끝

In [14]:
final_indices = []
for i in range(len(doc_indices)) :
    t_list = [doc_indices[i][result_indices[i][k]] for k in range(5)]
    final_indices.append(t_list)

In [15]:
total = []
for idx, example in enumerate(
        tqdm(dataset['validation'], desc="Dense retrieval: ")
    ):
        tmp = {
            # Query와 해당 id를 반환합니다.
            "question": example["question"],
            "id": example["id"],
            # Retrieve한 Passage의 id, context를 반환합니다.
            "context_id": final_indices[idx],
            "context": [corpus[pid] for pid in final_indices[idx]]
        }
        if "context" in example.keys() and "answers" in example.keys():
            # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
            tmp["original_context"] = example["context"]
            tmp["answers"] = example["answers"]
        total.append(tmp)

cqas_50 = pd.DataFrame(total)

In [24]:
cqas_50.to_csv('elastic_crossencoder.csv', index = False)

In [59]:
total = []
for idx, example in enumerate(
        tqdm(dataset['validation'], desc="Dense retrieval: ")
    ):
        tmp = {
            # Query와 해당 id를 반환합니다.
            "question": example["question"],
            "id": example["id"],
            # Retrieve한 Passage의 id, context를 반환합니다.
            "context_id": final_indices[idx],
            "context": " ".join(  # 기존에는 ' '.join()
                [corpus[pid] for pid in final_indices[idx]]
            ),
        }
        if "context" in example.keys() and "answers" in example.keys():
            # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
            tmp["original_context"] = example["context"]
            tmp["answers"] = example["answers"]
        total.append(tmp)

cqas_50 = pd.DataFrame(total)

In [ ]:
correct_length = []
for i in range(len(cqas_50)) :
    if cqas_50['original_context'][i] in cqas_50['context'][i] :
        correct_length.append(i)
print(len(correct_length) / len(dataset['validation']))

In [62]:
cqas_50.to_csv('test_b16_special_shuffle_elastic_ce40_t5.csv', index = False)